In [1]:
# to automatically update code as we go

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from csv import reader

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import lit
from pyspark.sql.functions import udf

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1547683576186_0005,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [58]:
research_data_2013_2014_2015 = sc.textFile("s3a://msds697-openpayments/OP_DTL_RSRCH_PGYR2013_P06292018.csv," + 
                                           "s3a://msds697-openpayments/OP_DTL_RSRCH_PGYR2014_P06292018.csv," + 
                                           "s3a://msds697-openpayments/OP_DTL_RSRCH_PGYR2015_P06292018.csv", 2)



In [59]:
research_data_2016_2017 = sc.textFile("s3a://msds697-openpayments/OP_DTL_RSRCH_PGYR2016_P06292018.csv," + 
                                      "s3a://msds697-openpayments/OP_DTL_RSRCH_PGYR2017_P06292018.csv")



In [60]:


def split_comma(x):
    return list(reader([x], delimiter=',', quotechar='"'))[0]

def check_columns(first_column, second_column):
    if first_column != '':
        return first_column
    else:
        return second_column

In [61]:
# loading in the data from 2013-2015 since they have the same columns
# however the data from 2016-2017 has slightly different columns
# checking that the columns are what we expect

columns_1 = research_data_2013_2014_2015.map(split_comma).take(1)
print(columns_1)

[['Change_Type', 'Covered_Recipient_Type', 'Noncovered_Recipient_Entity_Name', 'Teaching_Hospital_CCN', 'Teaching_Hospital_ID', 'Teaching_Hospital_Name', 'Physician_Profile_ID', 'Physician_First_Name', 'Physician_Middle_Name', 'Physician_Last_Name', 'Physician_Name_Suffix', 'Recipient_Primary_Business_Street_Address_Line1', 'Recipient_Primary_Business_Street_Address_Line2', 'Recipient_City', 'Recipient_State', 'Recipient_Zip_Code', 'Recipient_Country', 'Recipient_Province', 'Recipient_Postal_Code', 'Physician_Primary_Type', 'Physician_Specialty', 'Physician_License_State_code1', 'Physician_License_State_code2', 'Physician_License_State_code3', 'Physician_License_State_code4', 'Physician_License_State_code5', 'Principal_Investigator_1_Profile_ID', 'Principal_Investigator_1_First_Name', 'Principal_Investigator_1_Middle_Name', 'Principal_Investigator_1_Last_Name', 'Principal_Investigator_1_Name_Suffix', 'Principal_Investigator_1_Business_Street_Address_Line1', 'Principal_Investigator_1_Bu

In [62]:
# loading in the data from 2016-2017 since they have the same columns
# (with respect to eachother)
# checking that the columns are what we expect

columns_2 = research_data_2016_2017.map(split_comma).take(1)
print(columns_2)

[['Change_Type', 'Covered_Recipient_Type', 'Noncovered_Recipient_Entity_Name', 'Teaching_Hospital_CCN', 'Teaching_Hospital_ID', 'Teaching_Hospital_Name', 'Physician_Profile_ID', 'Physician_First_Name', 'Physician_Middle_Name', 'Physician_Last_Name', 'Physician_Name_Suffix', 'Recipient_Primary_Business_Street_Address_Line1', 'Recipient_Primary_Business_Street_Address_Line2', 'Recipient_City', 'Recipient_State', 'Recipient_Zip_Code', 'Recipient_Country', 'Recipient_Province', 'Recipient_Postal_Code', 'Physician_Primary_Type', 'Physician_Specialty', 'Physician_License_State_code1', 'Physician_License_State_code2', 'Physician_License_State_code3', 'Physician_License_State_code4', 'Physician_License_State_code5', 'Principal_Investigator_1_Profile_ID', 'Principal_Investigator_1_First_Name', 'Principal_Investigator_1_Middle_Name', 'Principal_Investigator_1_Last_Name', 'Principal_Investigator_1_Name_Suffix', 'Principal_Investigator_1_Business_Street_Address_Line1', 'Principal_Investigator_1_Bu

In [63]:
# checking for what columns the 2 different timeframes 
# (columns_1 (2013-2015) & columns_2 (2016-2017))
# have in common using sets

intersection = set(columns_1[0]).intersection(set(columns_2[0]))
print(intersection)

set(['Principal_Investigator_2_License_State_code3', 'Principal_Investigator_2_License_State_code2', 'Principal_Investigator_2_License_State_code1', 'Principal_Investigator_3_First_Name', 'Principal_Investigator_5_Business_Street_Address_Line2', 'Principal_Investigator_2_License_State_code5', 'Principal_Investigator_2_License_State_code4', 'Principal_Investigator_1_Middle_Name', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State', 'Recipient_Primary_Business_Street_Address_Line2', 'Principal_Investigator_4_Middle_Name', 'Recipient_Primary_Business_Street_Address_Line1', 'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name', 'Principal_Investigator_3_Name_Suffix', 'Principal_Investigator_2_Name_Suffix', 'Principal_Investigator_5_Middle_Name', 'Physician_Primary_Type', 'Principal_Investigator_4_Business_Street_Address_Line2', 'Principal_Investigator_4_Business_Street_Address_Line1', 'Principal_Investigator_5_Name_Suffix', 'Total_Amount_of_Payment_USDollars', 'Principal

In [64]:
# loading into a PySpark DataFrame (inferring from the schema to get it working quickly)

research_2016_2017_df = research_data_2016_2017.map(split_comma).toDF(columns_2[0])
research_2016_2017_df.show(5)

+-----------+----------------------+--------------------------------+---------------------+--------------------+----------------------+--------------------+--------------------+---------------------+-------------------+---------------------+-----------------------------------------------+-----------------------------------------------+--------------+---------------+------------------+-----------------+------------------+---------------------+----------------------+-------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------------+-----------------------------------+------------------------------------+----------------------------------+------------------------------------+------------------------------------------------------+------------------------------------------------------+-----------------------------+------------------------------+---------

In [10]:
# checking the row count

research_2016_2017_df.count()

1343102

In [65]:
# removing the column headers for 2016-2017

research_2016_2017_df = research_2016_2017_df.filter('Change_Type != "Change_Type"')

In [17]:
# double checking the row count

research_2016_2017_df.count()

1343100

In [19]:
# visually confirming the row is what we expect

research_2016_2017_df.take(2)

[Row(Change_Type=u'UNCHANGED', Covered_Recipient_Type=u'Covered Recipient Teaching Hospital', Noncovered_Recipient_Entity_Name=u'', Teaching_Hospital_CCN=u'050660', Teaching_Hospital_ID=u'4736', Teaching_Hospital_Name=u'USC NORRIS CANCER HOSPITAL', Physician_Profile_ID=u'', Physician_First_Name=u'', Physician_Middle_Name=u'', Physician_Last_Name=u'', Physician_Name_Suffix=u'', Recipient_Primary_Business_Street_Address_Line1=u'1441 Eastlake Ave', Recipient_Primary_Business_Street_Address_Line2=u'', Recipient_City=u'Los Angeles', Recipient_State=u'CA', Recipient_Zip_Code=u'90033', Recipient_Country=u'United States', Recipient_Province=u'', Recipient_Postal_Code=u'', Physician_Primary_Type=u'', Physician_Specialty=u'', Physician_License_State_code1=u'', Physician_License_State_code2=u'', Physician_License_State_code3=u'', Physician_License_State_code4=u'', Physician_License_State_code5=u'', Principal_Investigator_1_Profile_ID=u'240407', Principal_Investigator_1_First_Name=u'Eric', Princip

In [66]:
# loading into a PySpark DataFrame (inferring from the schema to get it working quickly)

research_2013_2015_df = research_data_2013_2014_2015.map(split_comma).toDF(columns_1[0])
research_2013_2015_df.show(5)

+-----------+----------------------+--------------------------------+---------------------+--------------------+----------------------+--------------------+--------------------+---------------------+-------------------+---------------------+-----------------------------------------------+-----------------------------------------------+--------------+---------------+------------------+-----------------+------------------+---------------------+----------------------+-------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------------+-----------------------------------+------------------------------------+----------------------------------+------------------------------------+------------------------------------------------------+------------------------------------------------------+-----------------------------+------------------------------+---------

In [67]:
# removing the column headers for 2013-2015

research_2013_2015_df = research_2013_2015_df.filter('Change_Type != "Change_Type"')

In [23]:
# double checking the row count

research_2013_2015_df.count()

2035191

In [25]:
research_2013_2015_df.filter('Form_of_Payment_or_Transfer_of_Value == "In-kind items and services"')\
                     .select('Total_Amount_of_Payment_USDollars', 'Form_of_Payment_or_Transfer_of_Value').show(20)

In [26]:
# counting rows that have no monetary value
# since they affect predictions

research_2013_2015_df.filter('Total_Amount_of_Payment_USDollars == 0.0').count()


In [68]:
# finding the columns that were in 2013-2015 but not in 2016-17

unique_2013_2014_2015 = set(columns_1[0]).difference(set(columns_2[0]))
print(unique_2013_2014_2015)

set(['Name_of_Associated_Covered_Drug_or_Biological2', 'Name_of_Associated_Covered_Drug_or_Biological3', 'Name_of_Associated_Covered_Drug_or_Biological1', 'Name_of_Associated_Covered_Drug_or_Biological4', 'Name_of_Associated_Covered_Drug_or_Biological5', 'NDC_of_Associated_Covered_Drug_or_Biological1', 'NDC_of_Associated_Covered_Drug_or_Biological3', 'NDC_of_Associated_Covered_Drug_or_Biological2', 'NDC_of_Associated_Covered_Drug_or_Biological5', 'NDC_of_Associated_Covered_Drug_or_Biological4', 'Name_of_Associated_Covered_Device_or_Medical_Supply4', 'Name_of_Associated_Covered_Device_or_Medical_Supply5', 'Name_of_Associated_Covered_Device_or_Medical_Supply2', 'Name_of_Associated_Covered_Device_or_Medical_Supply3', 'Name_of_Associated_Covered_Device_or_Medical_Supply1', 'Product_Indicator'])

In [69]:
# looking at the values of some of the columns to see 
# if they're similar to columns from the 2016-2017 dataset

research_2013_2015_df.select('Name_of_Associated_Covered_Drug_or_Biological1', 
                             'NDC_of_Associated_Covered_Drug_or_Biological1',
                             'Name_of_Associated_Covered_Device_or_Medical_Supply1',
                             'Product_Indicator').show(3)


+----------------------------------------------+---------------------------------------------+----------------------------------------------------+-----------------+
|Name_of_Associated_Covered_Drug_or_Biological1|NDC_of_Associated_Covered_Drug_or_Biological1|Name_of_Associated_Covered_Device_or_Medical_Supply1|Product_Indicator|
+----------------------------------------------+---------------------------------------------+----------------------------------------------------+-----------------+
|                                     HUMATROPE|                                   0002814801|                                                    |          Covered|
|                                       EFFIENT|                                   0002512330|                                                    |          Covered|
|                                       EFFIENT|                                   0002512330|                                                    |          Covered|
+---

In [70]:
# finding the columns that were in 2016-2017 but not in 2013-15

unique_2016_2017 = set(columns_2[0]).difference(set(columns_1[0]))
print(unique_2016_2017)

set(['Product_Category_or_Therapeutic_Area_1', 'Product_Category_or_Therapeutic_Area_2', 'Product_Category_or_Therapeutic_Area_3', 'Product_Category_or_Therapeutic_Area_4', 'Product_Category_or_Therapeutic_Area_5', 'Associated_Drug_or_Biological_NDC_1', 'Associated_Drug_or_Biological_NDC_3', 'Associated_Drug_or_Biological_NDC_2', 'Associated_Drug_or_Biological_NDC_5', 'Associated_Drug_or_Biological_NDC_4', 'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5', 'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4', 'Covered_or_Noncovered_Indicator_1', 'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1', 'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_3', 'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_2', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2', 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3', 'Name_of

In [71]:
# looking at the values of some of the columns to see 
# if they're similar to columns from the 2013-2015 dataset

research_2016_2017_df.select('Product_Category_or_Therapeutic_Area_1', 
                             'Associated_Drug_or_Biological_NDC_1',
                             'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1',
                             'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1',
                             'Covered_or_Noncovered_Indicator_1',
                             'Related_Product_Indicator').show(3)

+--------------------------------------+-----------------------------------+---------------------------------------------------------+--------------------------------------------------------+---------------------------------+-------------------------+
|Product_Category_or_Therapeutic_Area_1|Associated_Drug_or_Biological_NDC_1|Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1|Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1|Covered_or_Noncovered_Indicator_1|Related_Product_Indicator|
+--------------------------------------+-----------------------------------+---------------------------------------------------------+--------------------------------------------------------+---------------------------------+-------------------------+
|                             Radiology|                                   |                                                   Device|                                    STAR Tumor Ablati...|                          Covered|                   

In [72]:
modified_2013_2015_df = research_2013_2015_df.withColumn("Product_Category_or_Therapeutic_Area_1", lit("Not Field"))
modified_2013_2015_df = modified_2013_2015_df.withColumn('Product_Category_or_Therapeutic_Area_2', lit('Not Field'))
modified_2013_2015_df = modified_2013_2015_df.withColumn('Product_Category_or_Therapeutic_Area_3', lit('Not Field'))
modified_2013_2015_df = modified_2013_2015_df.withColumn('Product_Category_or_Therapeutic_Area_4', lit('Not Field'))
modified_2013_2015_df = modified_2013_2015_df.withColumn('Product_Category_or_Therapeutic_Area_5', lit('Not Field'))

modified_2013_2015_df = modified_2013_2015_df.withColumn('Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1', 
                                                         lit('Not Field'))
modified_2013_2015_df = modified_2013_2015_df.withColumn('Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_2', 
                                                         lit('Not Field'))
modified_2013_2015_df = modified_2013_2015_df.withColumn('Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_3', 
                                                         lit('Not Field'))
modified_2013_2015_df = modified_2013_2015_df.withColumn('Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4', 
                                                         lit('Not Field'))
modified_2013_2015_df = modified_2013_2015_df.withColumn('Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5', 
                                                         lit('Not Field'))

modified_2013_2015_df = modified_2013_2015_df.withColumn('Related_Product_Indicator', lit('Not Field'))
modified_2013_2015_df = modified_2013_2015_df.withColumn('type', lit('research'))


In [73]:
custom_udf = udf(check_columns, StringType())

modified_2013_2015_df = modified_2013_2015_df.withColumn('Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1', 
                                                         custom_udf(modified_2013_2015_df['Name_of_Associated_Covered_Drug_or_Biological1'],
                                                                    modified_2013_2015_df['Name_of_Associated_Covered_Device_or_Medical_Supply1']))


In [74]:
modified_2013_2015_df = modified_2013_2015_df.withColumn('Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2', 
                                                         custom_udf(modified_2013_2015_df['Name_of_Associated_Covered_Drug_or_Biological2'],
                                                                    modified_2013_2015_df['Name_of_Associated_Covered_Device_or_Medical_Supply2']))
modified_2013_2015_df = modified_2013_2015_df.withColumn('Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3', 
                                                         custom_udf(modified_2013_2015_df['Name_of_Associated_Covered_Drug_or_Biological3'],
                                                                    modified_2013_2015_df['Name_of_Associated_Covered_Device_or_Medical_Supply3']))
modified_2013_2015_df = modified_2013_2015_df.withColumn('Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4', 
                                                         custom_udf(modified_2013_2015_df['Name_of_Associated_Covered_Drug_or_Biological4'],
                                                                    modified_2013_2015_df['Name_of_Associated_Covered_Device_or_Medical_Supply4']))
modified_2013_2015_df = modified_2013_2015_df.withColumn('Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5', 
                                                         custom_udf(modified_2013_2015_df['Name_of_Associated_Covered_Drug_or_Biological5'],
                                                                    modified_2013_2015_df['Name_of_Associated_Covered_Device_or_Medical_Supply5']))

In [75]:
new_column_name = 'Drug_or_Biological_NDC_1'

modified_2013_2015_df = modified_2013_2015_df.withColumnRenamed('NDC_of_Associated_Covered_Drug_or_Biological1', 
                                                                new_column_name)

modified_2016_2017_df = research_2016_2017_df.withColumnRenamed('Associated_Drug_or_Biological_NDC_1', 
                                                               new_column_name)

modified_2016_2017_df = modified_2016_2017_df.withColumn('type', lit('research'))

In [76]:
# for columns that are similar in content (between the 2013-2015 and 2016-2017 data
# but are named differently, we created a new third column 

new_column_name_2 = 'Drug_or_Biological_NDC_2'

modified_2013_2015_df = modified_2013_2015_df.withColumnRenamed('NDC_of_Associated_Covered_Drug_or_Biological2', 
                                                                new_column_name_2)

modified_2016_2017_df = modified_2016_2017_df.withColumnRenamed('Associated_Drug_or_Biological_NDC_2', 
                                                               new_column_name_2)

new_column_name_3 = 'Drug_or_Biological_NDC_3'

modified_2013_2015_df = modified_2013_2015_df.withColumnRenamed('NDC_of_Associated_Covered_Drug_or_Biological3', 
                                                                new_column_name_3)

modified_2016_2017_df = modified_2016_2017_df.withColumnRenamed('Associated_Drug_or_Biological_NDC_3', 
                                                               new_column_name_3)
new_column_name_4 = 'Drug_or_Biological_NDC_4'

modified_2013_2015_df = modified_2013_2015_df.withColumnRenamed('NDC_of_Associated_Covered_Drug_or_Biological4', 
                                                                new_column_name_4)

modified_2016_2017_df = modified_2016_2017_df.withColumnRenamed('Associated_Drug_or_Biological_NDC_4', 
                                                               new_column_name_4)
new_column_name_5 = 'Drug_or_Biological_NDC_5'

modified_2013_2015_df = modified_2013_2015_df.withColumnRenamed('NDC_of_Associated_Covered_Drug_or_Biological5', 
                                                                new_column_name_5)

modified_2016_2017_df = modified_2016_2017_df.withColumnRenamed('Associated_Drug_or_Biological_NDC_5', 
                                                               new_column_name_5)

In [77]:
modified_2013_2015_df = modified_2013_2015_df.drop('Name_of_Associated_Covered_Device_or_Medical_Supply1', 
                                                   'Name_of_Associated_Covered_Device_or_Medical_Supply2',
                                                  'Name_of_Associated_Covered_Device_or_Medical_Supply3',
                                                  'Name_of_Associated_Covered_Device_or_Medical_Supply4',
                                                  'Name_of_Associated_Covered_Device_or_Medical_Supply5',
                                                  'Name_of_Associated_Covered_Drug_or_Biological1',
                                                  'Name_of_Associated_Covered_Drug_or_Biological2',
                                                  'Name_of_Associated_Covered_Drug_or_Biological3',
                                                  'Name_of_Associated_Covered_Drug_or_Biological4',
                                                  'Name_of_Associated_Covered_Drug_or_Biological5')

In [78]:
# for those columns that didn't exist in the 2013-2015 dataset 

modified_2013_2015_df = modified_2013_2015_df.withColumn('Covered_or_Noncovered_Indicator_1', lit('Not Field'))
modified_2013_2015_df = modified_2013_2015_df.withColumn('Covered_or_Noncovered_Indicator_2', lit('Not Field'))
modified_2013_2015_df = modified_2013_2015_df.withColumn('Covered_or_Noncovered_Indicator_3', lit('Not Field'))
modified_2013_2015_df = modified_2013_2015_df.withColumn('Covered_or_Noncovered_Indicator_4', lit('Not Field'))
modified_2013_2015_df = modified_2013_2015_df.withColumn('Covered_or_Noncovered_Indicator_5', lit('Not Field'))

modified_2016_2017_df = modified_2016_2017_df.withColumn('Product_Indicator', lit('Not Field'))

In [79]:
# confirming that there are no columns that are in the 2013-2015 dataset but not in the 2016-2017 dataset

print(set(modified_2013_2015_df.columns).difference(set(modified_2016_2017_df.columns)))

set([])

In [80]:
# confirming that there are no columns that are in the 2016-2017 dataset but not in the 2013-2015 dataset

print(set(modified_2016_2017_df.columns).difference(set(modified_2013_2015_df.columns)))

set([])

In [26]:
# checking how many rows are in each dataset

print(modified_2013_2015_df.count())
print(modified_2016_2017_df.count())

2035194
1470472

In [88]:
# at this point, all of the research data is standardized

total_research_df = modified_2013_2015_df.union(modified_2016_2017_df)


In [51]:
# verifying the number of rows matches the sum of the two previous numbers

print(total_research_df.count())

3505666

In [89]:
# confirming the number of columns

print(len(total_research_df.columns))

178

In [34]:
total_research_df.filter(total_research_df.Total_Amount_of_Payment_USDollars!=0.0).count()

2034828

In [ ]:
total_research_df.filter(total_research_df.Total_Amount_of_Payment_USDollars==0.0).count()

In [36]:
total_research_df.filter(total_research_df.Total_Amount_of_Payment_USDollars==0.0)\
                 .select('Form_of_Payment_or_Transfer_of_Value').distinct().show()


+------------------------------------+
|Form_of_Payment_or_Transfer_of_Value|
+------------------------------------+
|                Cash or cash equi...|
|                In-kind items and...|
+------------------------------------+

In [107]:
modified_total_research_df = total_research_df.drop('Physician_License_State_code2',
                                                   'Physician_License_State_code3',
                                                   'Physician_License_State_code4',
                                                   'Physician_License_State_code5',
                                                   'Teaching_Hospital_CCN',
                                                   'Payment_Publication_Date',
                                                   'Recipient_Postal_Code',
                                                   'Recipient_Province',
                                                    'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name', 
                                                    'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID', 
                                                    'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
                                                    'Principal_Investigator_1_First_Name', 
                                                    'Principal_Investigator_1_Middle_Name', 
                                                    'Principal_Investigator_1_Last_Name', 
                                                    'Principal_Investigator_1_Name_Suffix',
                                                    'Principal_Investigator_2_First_Name', 
                                                    'Principal_Investigator_2_Middle_Name', 
                                                    'Principal_Investigator_2_Last_Name', 
                                                    'Principal_Investigator_2_Name_Suffix',
                                                    'Principal_Investigator_3_First_Name', 
                                                    'Principal_Investigator_3_Middle_Name', 
                                                    'Principal_Investigator_3_Last_Name', 
                                                    'Principal_Investigator_3_Name_Suffix',
                                                    'Principal_Investigator_4_First_Name', 
                                                    'Principal_Investigator_4_Middle_Name', 
                                                    'Principal_Investigator_4_Last_Name', 
                                                    'Principal_Investigator_4_Name_Suffix',
                                                    'Principal_Investigator_5_First_Name', 
                                                    'Principal_Investigator_5_Middle_Name', 
                                                    'Principal_Investigator_5_Last_Name', 
                                                    'Principal_Investigator_5_Name_Suffix',
                                                    'Physician_First_Name', 
                                                    'Physician_Middle_Name', 
                                                    'Physician_Last_Name', 
                                                    'Physician_Name_Suffix',
                                                    'Recipient_Primary_Business_Street_Address_Line1', 
                                                    'Recipient_Primary_Business_Street_Address_Line2',
                                                    'Noncovered_Recipient_Entity_Name',
                                                    'Teaching_Hospital_Name',
                                                    'Name_of_Study',
                                                    'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1', 
                                                    'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2', 
                                                    'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3', 
                                                    'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4', 
                                                    'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5',
                                                    'Drug_or_Biological_NDC_1', 
                                                    'Drug_or_Biological_NDC_2', 
                                                    'Drug_or_Biological_NDC_3', 
                                                    'Drug_or_Biological_NDC_4', 
                                                    'Drug_or_Biological_NDC_5',
                                                    'Principal_Investigator_1_Business_Street_Address_Line1', 
                                                    'Principal_Investigator_1_Business_Street_Address_Line2',
                                                    'Principal_Investigator_2_Business_Street_Address_Line1', 
                                                    'Principal_Investigator_2_Business_Street_Address_Line2',
                                                    'Principal_Investigator_3_Business_Street_Address_Line1', 
                                                    'Principal_Investigator_3_Business_Street_Address_Line2',
                                                    'Principal_Investigator_4_Business_Street_Address_Line1', 
                                                    'Principal_Investigator_4_Business_Street_Address_Line2',
                                                    'Principal_Investigator_5_Business_Street_Address_Line1', 
                                                    'Principal_Investigator_5_Business_Street_Address_Line2'
                                                   )

In [108]:
print(len(modified_total_research_df.columns))

118

In [100]:
modified_total_research_df.columns

['Change_Type', 'Covered_Recipient_Type', 'Teaching_Hospital_ID', 'Physician_Profile_ID', 'Recipient_City', 'Recipient_State', 'Recipient_Zip_Code', 'Recipient_Country', 'Physician_Primary_Type', 'Physician_Specialty', 'Physician_License_State_code1', 'Principal_Investigator_1_Profile_ID', 'Principal_Investigator_1_Business_Street_Address_Line1', 'Principal_Investigator_1_Business_Street_Address_Line2', 'Principal_Investigator_1_City', 'Principal_Investigator_1_State', 'Principal_Investigator_1_Zip_Code', 'Principal_Investigator_1_Country', 'Principal_Investigator_1_Province', 'Principal_Investigator_1_Postal_Code', 'Principal_Investigator_1_Primary_Type', 'Principal_Investigator_1_Specialty', 'Principal_Investigator_1_License_State_code1', 'Principal_Investigator_1_License_State_code2', 'Principal_Investigator_1_License_State_code3', 'Principal_Investigator_1_License_State_code4', 'Principal_Investigator_1_License_State_code5', 'Principal_Investigator_2_Profile_ID', 'Principal_Investi

In [109]:
subset = modified_total_research_df.limit(100)

In [110]:
subset.coalesce(1).write.save("s3a://msds697-openpayments/subset_research_100", format='csv', header=True)


In [ ]:
modified_total_research_df.coalesce(1).write.save("s3a://msds697-openpayments/total_research_unfiltered",
                                                  format='csv', header=True)


In [44]:
modified_total_research_df.coalesce(1).write.save("s3a://msds697-openpayments/total_research_unfiltered",
                                                  format='csv', header=True)


In [45]:
# filter to remove rows with 0 dollars

filtered_total_research_df = modified_total_research_df.filter(total_research_df.Total_Amount_of_Payment_USDollars != 0.0)
print(filtered_total_research_df.count())

2034828

In [46]:
modified_total_research_df.coalesce(1).write.save("s3a://msds697-openpayments/total_research_filtered",
                                                  format='csv', header=True)


In [30]:
print(len(total_research_df.columns))

177

In [31]:
print(total_research_df.columns)

['Change_Type', 'Covered_Recipient_Type', 'Noncovered_Recipient_Entity_Name', 'Teaching_Hospital_CCN', 'Teaching_Hospital_ID', 'Teaching_Hospital_Name', 'Physician_Profile_ID', 'Physician_First_Name', 'Physician_Middle_Name', 'Physician_Last_Name', 'Physician_Name_Suffix', 'Recipient_Primary_Business_Street_Address_Line1', 'Recipient_Primary_Business_Street_Address_Line2', 'Recipient_City', 'Recipient_State', 'Recipient_Zip_Code', 'Recipient_Country', 'Recipient_Province', 'Recipient_Postal_Code', 'Physician_Primary_Type', 'Physician_Specialty', 'Physician_License_State_code1', 'Physician_License_State_code2', 'Physician_License_State_code3', 'Physician_License_State_code4', 'Physician_License_State_code5', 'Principal_Investigator_1_Profile_ID', 'Principal_Investigator_1_First_Name', 'Principal_Investigator_1_Middle_Name', 'Principal_Investigator_1_Last_Name', 'Principal_Investigator_1_Name_Suffix', 'Principal_Investigator_1_Business_Street_Address_Line1', 'Principal_Investigator_1_Bus

In [49]:
# to unify the ownership files across the different years

ownership_2013_2017 = sc.textFile("s3a://msds697-openpayments/OP_DTL_OWNRSHP_PGYR2017_P06292018.csv," + 
                                  "s3a://msds697-openpayments/OP_DTL_OWNRSHP_PGYR2016_P06292018.csv," + 
                                  "s3a://msds697-openpayments/OP_DTL_OWNRSHP_PGYR2015_P06292018.csv," + 
                                  "s3a://msds697-openpayments/OP_DTL_OWNRSHP_PGYR2014_P06292018.csv," + 
                                  "s3a://msds697-openpayments/OP_DTL_OWNRSHP_PGYR2013_P06292018.csv", 2)



In [50]:
columns_3 = ownership_2013_2017.map(split_comma).take(1)
print(columns_3)

[['Change_Type', 'Physician_Profile_ID', 'Physician_First_Name', 'Physician_Middle_Name', 'Physician_Last_Name', 'Physician_Name_Suffix', 'Recipient_Primary_Business_Street_Address_Line1', 'Recipient_Primary_Business_Street_Address_Line2', 'Recipient_City', 'Recipient_State', 'Recipient_Zip_Code', 'Recipient_Country', 'Recipient_Province', 'Recipient_Postal_Code', 'Physician_Primary_Type', 'Physician_Specialty', 'Record_ID', 'Program_Year', 'Total_Amount_Invested_USDollars', 'Value_of_Interest', 'Terms_of_Interest', 'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country', 'Dispute_Status_for_Publication', 'Interest_Held_by_Physician_or_an_Immediate_Family_Member', 'Payment_Publication_Date']]

In [51]:
print(len(columns_3))

1

In [11]:
test = sc.textFile('s3a://msds697-openpayments/subset_test_1/' + 
                   'part-00000-65c52fec-3de1-4643-9516-fa329ff684b2-c000.csv', 
                   2)


In [10]:
test.take(1)

An error occurred while calling o188.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: s3://msds697-openpayments/subset_test_1/part-00000-65c52fec-3de1-4643-9516-fa329ff684b2-c000.csv
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:260)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:208)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:288)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:204)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$

In [16]:
ss = SparkSession.builder.getOrCreate()

In [17]:
data = ss.read.csv('s3a://msds697-openpayments/subset_test_1/part-00000-65c52fec-3de1-4643-9516-fa329ff684b2-c000.csv')


An error occurred while calling o218.csv.
: java.nio.file.AccessDeniedException: s3a://msds697-openpayments/subset_test_1/part-00000-65c52fec-3de1-4643-9516-fa329ff684b2-c000.csv: getFileStatus on s3a://msds697-openpayments/subset_test_1/part-00000-65c52fec-3de1-4643-9516-fa329ff684b2-c000.csv: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: 66E1899044A2A03A; S3 Extended Request ID: i+bZtmt+P0bsrF01jc3+3tP3WBois5h9583Hl1Tk851iQwxFSJPbmgjUDGsUZyYZEMz4JSZdJkc=), S3 Extended Request ID: i+bZtmt+P0bsrF01jc3+3tP3WBois5h9583Hl1Tk851iQwxFSJPbmgjUDGsUZyYZEMz4JSZdJkc=
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:158)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:101)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:1568)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:117)
	at org.apache.hadoop.fs.Fil

In [18]:
subset_uploaded_file = 's3a://msds697-openpayments/subset_test_1/' + \
                       'part-00000-65c52fec-3de1-4643-9516-fa329ff684b2-c000.csv'
subset_uploaded = spark.read.format("csv").option("header", 
                                                  "true")\
                       .option("encoding", "utf-8").load(subset_uploaded_file)


An error occurred while calling o255.load.
: java.nio.file.AccessDeniedException: s3a://msds697-openpayments/subset_test_1/part-00000-65c52fec-3de1-4643-9516-fa329ff684b2-c000.csv: getFileStatus on s3a://msds697-openpayments/subset_test_1/part-00000-65c52fec-3de1-4643-9516-fa329ff684b2-c000.csv: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: 521BF5D9E0989FA8; S3 Extended Request ID: v/UV4/cDsYZssVrauIphjWHXkFsMXDhCHzQEnVFCL9FQZSV/PnZ8zlld9oO3fqM1u0/JnYgusrA=), S3 Extended Request ID: v/UV4/cDsYZssVrauIphjWHXkFsMXDhCHzQEnVFCL9FQZSV/PnZ8zlld9oO3fqM1u0/JnYgusrA=
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:158)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:101)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:1568)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:117)
	at org.apache.hadoop.fs.Fi

In [19]:
subset_file = 's3://msds697-openpayments/subset_test_2/part-00000-0010f144-ce79-41ee-aad3-473a76d0a6b6-c000.csv'
subset_uploaded = spark.read.format("csv").option("header", "true")\
                       .option("encoding", "utf-8").load(subset_uploaded_file)

An error occurred while calling o290.load.
: java.nio.file.AccessDeniedException: s3a://msds697-openpayments/subset_test_1/part-00000-65c52fec-3de1-4643-9516-fa329ff684b2-c000.csv: getFileStatus on s3a://msds697-openpayments/subset_test_1/part-00000-65c52fec-3de1-4643-9516-fa329ff684b2-c000.csv: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: 4779FE454C81AF93; S3 Extended Request ID: 0pdu64V+Lpq1uKzO/Ahlxwyn/01Y98vIeXLU25Ug7RIISweNlQKrXRRzgWRBrgBlCZ+eDqjH/vI=), S3 Extended Request ID: 0pdu64V+Lpq1uKzO/Ahlxwyn/01Y98vIeXLU25Ug7RIISweNlQKrXRRzgWRBrgBlCZ+eDqjH/vI=
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:158)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:101)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:1568)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:117)
	at org.apache.hadoop.fs.Fi